In [ ]:
# linear algebra
import numpy as np
# data processing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import datetime
from IPython.display import display, HTML
%matplotlib inline


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Specify file path
celeb_attr = '/content/drive/MyDrive/celeb/list_attr_celeba.txt'

#Read CSV file using pandas
import pandas as pd
celeb_attr_df = pd.read_csv(celeb_attr)

# Display the first few rows of the DataFrame
print(celeb_attr_df.head())


                                              202599
0  5_o_Clock_Shadow Arched_Eyebrows Attractive Ba...
1  000001.jpg -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1...
2  000002.jpg -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  1...
3  000003.jpg -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1...
4  000004.jpg -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1...


In [ ]:
# Specify file path
anno = '/content/drive/MyDrive/celeb/list_attr_celeba.txt'

#Read CSV file using pandas
import pandas as pd
anno_df = pd.read_csv(anno)

# Display the first few rows of the DataFrame
print(anno_df.head())



                                              202599
0  5_o_Clock_Shadow Arched_Eyebrows Attractive Ba...
1  000001.jpg -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1...
2  000002.jpg -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  1...
3  000003.jpg -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1...
4  000004.jpg -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1...


In [ ]:
# read in the partitions to discover train, validation and test segments
# Specify file path
eval_part = '/content/drive/MyDrive/celeb/list_eval_partition.txt'

#Read CSV file using pandas
import pandas as pd
eval_part_df = pd.read_csv(eval_part)

# Display the first few rows of the DataFrame
print(eval_part_df.head())


   000001.jpg 0
0  000002.jpg 0
1  000003.jpg 0
2  000004.jpg 0
3  000005.jpg 0
4  000006.jpg 0


In [ ]:
# read in the bounding boxes - these will be used to crop the images
# read in the partitions to discover train, validation and test segments
# Specify file path
box_celeb = '/content/drive/MyDrive/celeb/list_bbox_celeba.txt'

#Read CSV file using pandas
import pandas as pd
box_celeb_df = pd.read_csv(box_celeb)

# Display the first few rows of the DataFrame
print(box_celeb_df.head())


                          202599
0  image_id x_1 y_1 width height
1   000001.jpg    95  71 226 313
2   000002.jpg    72  94 221 306
3   000003.jpg   216  59  91 126
4   000004.jpg   622 257 564 781


In [ ]:
anno_df['image_id'] = range(len(anno_df))
box_celeb_df['image_id'] = range(len(box_celeb_df))
eval_part_df['image_id'] = range(len(eval_part_df))

# perform an outer join of the anno and bbox data frames on image_id
merged_df = pd.merge(anno_df, box_celeb_df, on='image_id', how='outer')

# perform an outer join of the result with the partition data frame on image_id to obtain integrated partitions
integrated_partitions = pd.merge(merged_df, eval_part_df, on='image_id', how='outer')
print(integrated_partitions.head(5))


                                            202599_x  image_id  \
0  5_o_Clock_Shadow Arched_Eyebrows Attractive Ba...         0   
1  000001.jpg -1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1...         1   
2  000002.jpg -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  1...         2   
3  000003.jpg -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1...         3   
4  000004.jpg -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1...         4   

                        202599_y  000001.jpg 0  
0  image_id x_1 y_1 width height  000002.jpg 0  
1   000001.jpg    95  71 226 313  000003.jpg 0  
2   000002.jpg    72  94 221 306  000004.jpg 0  
3   000003.jpg   216  59  91 126  000005.jpg 0  
4   000004.jpg   622 257 564 781  000006.jpg 0  


In [ ]:
# Get the rows for each partition
train_df = integrated_partitions[integrated_partitions['image_id'] == 0]
test_df = integrated_partitions[integrated_partitions['image_id'] == 1]
val_df = integrated_partitions[integrated_partitions['image_id'] == 2]


In [ ]:
# randomly select a fraction of images from the train dataframe
import numpy as np

# Set the fraction of images to select
frac = 0.1

# Get the number of images to select
num_images = int(np.round(frac * len(train_df)))

# Randomly select the images
random_train_df = train_df.sample(n=num_images, random_state=42)
random_train_df.head()


,202599_x,image_id,202599_y,000001.jpg 0


In [ ]:
# similarly select  fraction of images from the validation dataframe
import numpy as np

# Set the fraction of images to select
frac = 0.1

# Get the number of images to select
num_images = int(np.round(frac * len(val_df)))

# Randomly select the images
random_val_df = val_df.sample(n=num_images, random_state=42)

random_val_df.head()

,202599_x,image_id,202599_y,000001.jpg 0


In [ ]:
# Data generator class - this is a key class that is used to batch the data so as to
# reduce compute time as well as to fit training segments into available memory
# Additionally it allows you to specify multiple targets for classification
# Also allows for image cropping
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
class DataGenerator(Sequence):
    def __init__(self, df, batch_size=32, dim=(64,64), n_channels=3, n_classes=3, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # saves memory by batching
        df_temp = self.df.iloc[indexes]
        X, y = self.__data_generation(df_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, df_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)
        for i, row in enumerate(df_temp.values):
            img = image.load_img('celeba-dataset/img_align_celeba/img_align_celeba/' + row[0]) # extract image
            #crop the iamges using the bounding boxes
            def __data_generation(self, batch_df):
                   X = np.empty((self.batch_size, *self.dim, self.n_channels))
                   y = np.empty((self.batch_size, self.n_classes))

                   for i, row in batch_df.iterrows():
                       img = load_img(row['image_path'], target_size=None)
                       img = img_to_array(img)

                       # Get bounding box coordinates
                       x1, y1, x2, y2 = row['xmin'], row['ymin'], row['xmax'], row['ymax']

                       # Crop image using bounding box
                       img_cropped = img[int(y1):int(y2), int(x1):int(x2), :]

                       #normalize the cropped and resized images
                       # Resize image to desired dimensions
                       img_resized = cv2.resize(img_cropped, self.dim)

                       X[i,] = img_resized / 255.0
                       y[i,] = row['class_label']
                       y1[i,] = row['class_label1']
                       y2[i,] = row['class_label2']

            return X, [y1, y2]



In [ ]:
# using vgg16 as feature extractor - this is a simpler, but equally eefective
# version of the code used for the Dogs vs Cats example
import tensorflow as tf
vgg16 = tf.keras.applications.VGG16(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
vgg16.trainable = False

In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of base model
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 5
batch_size = 16

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")

for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True

last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


In [ ]:

train_df, validate_df = train_test_split(integrated_partitions, test_size=0.1)
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()

# validate_df = validate_df.sample(n=100).reset_index() # use for fast testing code purpose
# train_df = train_df.sample(n=1800).reset_index() # use for fast testing code purpose

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [ ]:
# creating the train, test and validation data generators
train_generator = DataGenerator(train_df, batch_size=32, dim=(64,64), n_channels=3, n_classes=3, shuffle=True)
test_generator = DataGenerator(test_df, batch_size=32, dim=(64,64), n_channels=3, n_classes=3, shuffle=True)
val_generator = DataGenerator(validate_df, batch_size=32, dim=(64,64), n_channels=3, n_classes=3, shuffle=True)

In [ ]:
import tensorflow as tf
img = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/celeba/celeb/img_align_celeba/060202.jpg')
img

OSError: ignored

In [ ]:
# training the model
from tensorflow.keras.preprocessing import image
# fine-tune the model
#history = model.fit(train_generator,epochs=1,validation_data=val_generator,validation_steps=total_validate/batch_size)
history = model.fit(train_generator,epochs=1, validation_data=val_generator)


TypeError: ignored

In [ ]:
# plotting the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: ignored

# **R1**

In [ ]:
import pandas as pd

class_names = ['image_id', 'Smiling', 'Young', 'Male', 'Oval_Face']

df_attr = pd.read_csv('/content/drive/MyDrive/celeba/celeb/list_attr_celeba.csv')

df_attr = df_attr[class_names]

# Filter
df_attr = df_attr.loc[(df_attr['Smiling'] == 1)
            | (df_attr['Young'] == 1)
            | (df_attr['Male'] == 1)
            | (df_attr['Oval_Face'] == 1) ]

df_attr.head(5)

,image_id,Smiling,Young,Male,Oval_Face
0,000001.jpg,1,1,-1,-1
1,000002.jpg,1,1,-1,-1
2,000003.jpg,-1,1,1,-1
3,000004.jpg,-1,1,-1,-1
4,000005.jpg,-1,1,-1,-1


In [ ]:
from sklearn.model_selection import train_test_split

train_df, validate_df = train_test_split(df_attr, test_size=0.1)

total_train_images = train_df.shape[0]
total_validate_images = validate_df.shape[0]

total_train_images, total_validate_images

(178626, 19848)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128

IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
BATCH_SIZE = 16

EPOCHS = 1

train_image_data_generator = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_image_data_generator.flow_from_dataframe(
    dataframe=train_df,
    directory="/content/drive/MyDrive/celeba/celeb/img_align_celeba/",
    x_col='image_id',
    y_col=class_names[1:],
    class_mode='raw',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
validate_image_data_generator = ImageDataGenerator(rescale=1./255)

validate_generator = validate_image_data_generator.flow_from_dataframe(
    dataframe=validate_df,
    directory="/content/drive/MyDrive/celeba/celeb/img_align_celeba/",
    x_col='image_id',
    y_col=class_names[1:],
    class_mode='raw',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

NameError: ignored

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

NameError: ignored

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 1
batch_size = 16

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")

for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True

last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validate_generator,
    validation_steps=total_validate_images/BATCH_SIZE)

NameError: ignored

# **R2**

In [ ]:
# Reading landmarks & calculating the mouth_width

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/celeba/celeb/list_landmarks_align_celeba.csv')

df = df[['image_id', 'leftmouth_x', 'rightmouth_x']]

df['mouth_width'] = df['rightmouth_x'] - df['leftmouth_x']

df.head()

,image_id,leftmouth_x,rightmouth_x,mouth_width
0,000001.jpg,73,108,35
1,000002.jpg,70,108,38
2,000003.jpg,74,98,24
3,000004.jpg,71,101,30
4,000005.jpg,71,104,33


In [ ]:
# Normalize normalize the mouth_width fixed range [0..1]

from sklearn.preprocessing import MinMaxScaler

df['mouth_width_norm'] = pd.DataFrame(MinMaxScaler().fit_transform(df[['mouth_width']]), columns=['mouth_width'])

df.head()

,image_id,leftmouth_x,rightmouth_x,mouth_width,mouth_width_norm
0,000001.jpg,73,108,35,0.557377
1,000002.jpg,70,108,38,0.606557
2,000003.jpg,74,98,24,0.377049
3,000004.jpg,71,101,30,0.475410
4,000005.jpg,71,104,33,0.524590


In [ ]:
df['mouth_width_bin_cat'] = pd.cut(df['mouth_width_norm'], bins=[0.5, 1.0], right=True)

df.head()

,image_id,leftmouth_x,rightmouth_x,mouth_width,mouth_width_norm,mouth_width_bin_cat
0,000001.jpg,73,108,35,0.557377,"(0.5, 1.0]"
1,000002.jpg,70,108,38,0.606557,"(0.5, 1.0]"
2,000003.jpg,74,98,24,0.377049,NaN
3,000004.jpg,71,101,30,0.475410,NaN
4,000005.jpg,71,104,33,0.524590,"(0.5, 1.0]"


In [ ]:
df['mouth_width_bin'] = 1

df.loc[df['mouth_width_bin_cat'].isnull(), 'mouth_width_bin'] = -1

df.head()

,image_id,leftmouth_x,rightmouth_x,mouth_width,mouth_width_norm,mouth_width_bin_cat,mouth_width_bin
0,000001.jpg,73,108,35,0.557377,"(0.5, 1.0]",1
1,000002.jpg,70,108,38,0.606557,"(0.5, 1.0]",1
2,000003.jpg,74,98,24,0.377049,NaN,-1
3,000004.jpg,71,101,30,0.475410,NaN,-1
4,000005.jpg,71,104,33,0.524590,"(0.5, 1.0]",1


In [ ]:
df.to_csv('mouth_width_bin_celebA')

In [ ]:
df.drop(columns=['mouth_width', 'leftmouth_x', 'rightmouth_x', 'mouth_width_norm', 'mouth_width_bin_cat'])

,image_id,mouth_width_bin
0,000001.jpg,1
1,000002.jpg,1
2,000003.jpg,-1
3,000004.jpg,-1
4,000005.jpg,1
...,...,...
202594,202595.jpg,-1
202595,202596.jpg,1
202596,202597.jpg,1
202597,202598.jpg,1


# **R3**

In [ ]:
import pandas as pd
import numpy as np

# Load the CelebA attribute annotations
df_attr = pd.read_csv('/content/drive/MyDrive/celeba/celeb/list_attr_celeba.csv')

# Extract the relevant attributes and preprocess them
df_attr = df_attr[['image_id', 'Young', 'Male', 'Oval_Face', 'Mouth_Slightly_Open']]
df_attr.replace(to_replace=-1, value=0, inplace=True)
#df_attr['Mouth_Wide_Open'] = np.where(df_attr['Mouth_Wide_Open'] == 1, 1, 0)

# Load and preprocess the images Reading landmarks & calculating the mouth_width

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/celeba/celeb/list_landmarks_align_celeba.csv')

df = df[['image_id', 'leftmouth_x', 'rightmouth_x']]

df['mouth_width'] = df['rightmouth_x'] - df['leftmouth_x']

df.head()


,image_id,leftmouth_x,rightmouth_x,mouth_width
0,000001.jpg,73,108,35
1,000002.jpg,70,108,38
2,000003.jpg,74,98,24
3,000004.jpg,71,101,30
4,000005.jpg,71,104,33


In [ ]:
y_train = df_attr[['Young', 'Male', 'Oval_Face', 'Mouth_Slightly_Open']].values
y_val = df[['image_id', 'leftmouth_x', 'rightmouth_x']].values


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Load the test set
test_generator = datagen.flow_from_dataframe(
    dataframe=df_attr,
    directory='/content/drive/MyDrive/celeba/celeb/img_align_celeba',
    x_col='image_id',
    y_col=['Young', 'Male', 'Oval_Face', 'Mouth_Slightly_Open'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the pre-trained VGG16 model and add custom layers for classification
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = vgg16_base.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='sigmoid')(x)
model = Model(inputs=vgg16_base.input, outputs=predictions)

# Compile the model and set the optimizer and loss function
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training set and validate on the validation set
model.fit_generator(
    generator=train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate_generator(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)


TypeError: ignored

In [ ]:
# plotting the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: ignored

# **R4**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.metrics import accuracy_score

# Load the CelebA dataset
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        'CelebA/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        classes=['male_young_oval_smiling']
)
test_generator = datagen.flow_from_directory(
        'CelebA/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        classes=['male_young_oval_smiling']
)

# Load the VGG16 pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune the last few layers of the model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training set
model.fit(
        train_generator,
        epochs=10,
        validation_data=test_generator
)

# Evaluate the accuracy of the model on the test set
test_images, test_labels = next(test_generator)
test_preds = model.predict(test_images)
test_preds = np.round(test_preds).flatten()
test_accuracy = accuracy_score(test_labels, test_preds)
print('Test accuracy:', test_accuracy)

# Retrieve images with young males with oval faces who are smiling using query R3
test_generator_smiling = datagen.flow_from_directory(
        'CelebA/test',
        target_size=(224, 224),
        batch_size=1,
        class_mode='binary',
        classes=['male_young_oval']
)
test_images_smiling, test_labels_smiling = next(test_generator_smiling)
test_preds_smiling = model.predict(test_images_smiling)
test_preds_smiling = np.round(test_preds_smiling).flatten()
index = np.where((test_preds_smiling == 1))[0][2] # index of the 3rd image that satisfies query R3
image = plt.imread('CelebA/test/male_young_oval/{}.jpg'.format(index+1))
plt.imshow(image)
plt.show()

# Compare the effectiveness of using mouth_width in place of the smiling annotation
test_generator_mouth = datagen.flow_from_directory


Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


ValueError: ignored